# Part 1.5: Optimization & Linear Programming

Every machine learning algorithm is, at its core, an optimization problem. When we train a neural network, we are searching for the set of weights that minimizes a loss function. When we fit a support vector machine, we are solving a constrained quadratic program. When we tune hyperparameters, we are optimizing a noisy black-box function.

Understanding optimization gives you the vocabulary and intuition to answer questions like:
- *Why does gradient descent work?* (Because loss functions are often convex, or at least locally convex.)
- *Why does regularization help?* (It adds a constraint to the optimization, shrinking the feasible region.)
- *Why do SVMs find the "best" boundary?* (They solve a convex optimization problem with a unique global solution.)

This notebook builds the mathematical foundation you need before diving into gradient-based methods in the next notebook.

## Learning Objectives

By the end of this notebook, you should be able to:

- [ ] Define an optimization problem (objective, constraints, feasible region)
- [ ] Formulate and solve linear programs graphically and with `scipy`
- [ ] Explain the Simplex method at a conceptual level
- [ ] State weak and strong duality and explain their significance
- [ ] Distinguish convex from non-convex problems and explain why convexity matters
- [ ] Apply Lagrange multipliers to equality-constrained problems
- [ ] State the KKT conditions and explain their role in inequality constraints
- [ ] Recognize ML algorithms as optimization problems (SVM, regularized regression)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog, minimize
from matplotlib.patches import Polygon
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

---

## 1. What is Optimization?

### Intuitive Explanation

Optimization is the art and science of finding the **best** solution from a set of possible solutions. In everyday life, you optimize constantly:
- Choosing the fastest route to work (minimize travel time)
- Allocating a budget across expenses (maximize value subject to spending limits)
- Setting a thermostat (minimize energy use while staying comfortable)

In mathematics, an optimization problem has three parts:

1. **Objective function** $f(x)$: The quantity we want to minimize (or maximize)
2. **Decision variables** $x$: The knobs we can turn
3. **Constraints**: Rules that limit which values of $x$ are allowed

The general form is:

$$\min_{x} f(x) \quad \text{subject to} \quad g_i(x) \leq 0, \quad h_j(x) = 0$$

The set of all $x$ that satisfy the constraints is called the **feasible region**. The best point within the feasible region is the **optimal solution**.

**What this means:** Think of the objective function as a landscape of hills and valleys. The constraints fence off a region you are allowed to explore. Your job is to find the lowest valley (for minimization) inside the fence.

In [ ]:
# Visualize a simple 2D optimization landscape
def objective_landscape(x1, x2):
    """A quadratic objective: f(x1, x2) = (x1 - 2)^2 + (x2 - 1)^2"""
    return (x1 - 2)**2 + (x2 - 1)**2

x1 = np.linspace(-1, 5, 300)
x2 = np.linspace(-1, 4, 300)
X1, X2 = np.meshgrid(x1, x2)
Z = objective_landscape(X1, X2)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: contour plot
ax = axes[0]
contour = ax.contourf(X1, X2, Z, levels=20, cmap='Blues_r', alpha=0.7)
ax.contour(X1, X2, Z, levels=20, colors='navy', alpha=0.3, linewidths=0.5)
ax.plot(2, 1, 'r*', markersize=15, label='Optimal point (2, 1)')
plt.colorbar(contour, ax=ax, label='f(x₁, x₂)')
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title('Unconstrained Optimization Landscape')
ax.legend()

# Right: same landscape with a constraint region
ax = axes[1]
contour = ax.contourf(X1, X2, Z, levels=20, cmap='Blues_r', alpha=0.7)
ax.contour(X1, X2, Z, levels=20, colors='navy', alpha=0.3, linewidths=0.5)

# Constraint: x1 + x2 <= 2 and x1 >= 0 and x2 >= 0
feasible_vertices = np.array([[0, 0], [2, 0], [0, 2]])
feasible_region = Polygon(feasible_vertices, alpha=0.3, color='green', label='Feasible region')
ax.add_patch(feasible_region)
ax.plot([0, 2], [2, 0], 'g-', linewidth=2)
ax.plot([0, 0], [0, 2], 'g-', linewidth=2)
ax.plot([0, 2], [0, 0], 'g-', linewidth=2)

# The constrained optimum is at (1, 1) — closest feasible point to (2,1)
# Actually, minimize (x1-2)^2 + (x2-1)^2 s.t. x1+x2<=2, x1>=0, x2>=0
# Optimal is at (1, 1) on the boundary x1+x2=2
ax.plot(2, 1, 'r*', markersize=12, alpha=0.3, label='Unconstrained optimum')
ax.plot(1, 1, 'r^', markersize=12, label='Constrained optimum (1, 1)')
ax.annotate('', xy=(1, 1), xytext=(2, 1),
            arrowprops=dict(arrowstyle='->', color='red', lw=1.5, ls='--'))
plt.colorbar(contour, ax=ax, label='f(x₁, x₂)')
ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title('Constrained Optimization')
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

print("Left: Without constraints, the minimum is at (2, 1) where f = 0.")
print("Right: With the constraint x₁ + x₂ ≤ 2 (and x₁, x₂ ≥ 0),")
print("       the minimum shifts to (1, 1) on the constraint boundary, where f = 1.")

### Deep Dive: Types of Optimization Problems

Optimization problems are classified by the nature of their objective and constraints:

| Type | Objective | Constraints | Example in ML |
|------|-----------|-------------|---------------|
| **Linear Programming (LP)** | Linear | Linear inequalities/equalities | Resource allocation, network flow |
| **Quadratic Programming (QP)** | Quadratic | Linear | SVM (dual form), ridge regression |
| **Convex Optimization** | Convex | Convex set | Logistic regression, LASSO |
| **Non-convex Optimization** | Non-convex | Any | Neural network training |
| **Integer Programming** | Linear/Quadratic | Integer variables | Feature selection |

#### Key Insight

The difficulty of an optimization problem depends on its **structure**, not its size. A convex problem with a million variables is easier (in principle) than a non-convex problem with ten variables. This is why understanding convexity matters so much in ML.

---

## 2. Linear Programming

### Intuitive Explanation

A **linear program (LP)** is the simplest type of optimization problem: both the objective function and all constraints are linear.

**Standard form:**

$$\min_{x} \mathbf{c}^T \mathbf{x} \quad \text{subject to} \quad A\mathbf{x} \leq \mathbf{b}, \quad \mathbf{x} \geq 0$$

#### Breaking down the formula:

| Component | Meaning | Role |
|-----------|---------|------|
| $\mathbf{c}^T \mathbf{x}$ | Dot product of cost vector and decision variables | Objective to minimize |
| $A\mathbf{x} \leq \mathbf{b}$ | Matrix inequality | Constraint on resources |
| $\mathbf{x} \geq 0$ | Non-negativity | Variables must be non-negative |

**What this means:** Imagine you run a factory making two products. Each product gives you a different profit (the objective). Each product uses different amounts of labor and materials (the constraints). How much of each product should you make to maximize profit?

### Geometric Intuition

The feasible region of an LP is always a **convex polytope** (a polygon in 2D, a polyhedron in 3D, etc.). The objective function creates parallel hyperplanes across the space. The optimal solution always occurs at a **vertex** of the polytope.

Why a vertex? Because a linear function over a convex region achieves its extreme values at the boundary, and specifically at a corner point.

In [ ]:
# Example LP: A factory production problem
# Maximize: 5*x1 + 4*x2 (profit)
# Subject to:
#   6*x1 + 4*x2 <= 24  (labor hours)
#   x1 + 2*x2 <= 6     (raw material)
#   x1, x2 >= 0

# Graphical solution
fig, ax = plt.subplots(figsize=(10, 8))

x = np.linspace(0, 5, 400)

# Constraint boundaries
y1 = (24 - 6*x) / 4   # 6x1 + 4x2 = 24
y2 = (6 - x) / 2       # x1 + 2x2 = 6

# Plot constraints
ax.plot(x, y1, 'b-', linewidth=2, label='6x₁ + 4x₂ ≤ 24 (labor)')
ax.plot(x, y2, 'orange', linewidth=2, label='x₁ + 2x₂ ≤ 6 (material)')
ax.axhline(y=0, color='k', linewidth=0.8)
ax.axvline(x=0, color='k', linewidth=0.8)

# Feasible region vertices (found by solving intersection equations)
vertices = np.array([
    [0, 0],
    [4, 0],    # x1=4, x2=0: from 6*4 + 4*0 = 24
    [3, 1.5],  # intersection of 6x1+4x2=24 and x1+2x2=6
    [0, 3],    # x1=0, x2=3: from 0 + 2*3 = 6
])

# Fill feasible region
feasible = Polygon(vertices, alpha=0.2, color='green', label='Feasible region')
ax.add_patch(feasible)

# Mark vertices
for v in vertices:
    profit = 5*v[0] + 4*v[1]
    ax.plot(v[0], v[1], 'ko', markersize=8)
    ax.annotate(f'({v[0]:.1f}, {v[1]:.1f})\nProfit = {profit:.1f}',
                xy=(v[0], v[1]), xytext=(v[0]+0.15, v[1]+0.2),
                fontsize=9, fontweight='bold')

# Optimal vertex
ax.plot(3, 1.5, 'r*', markersize=20, zorder=5, label='Optimal: (3, 1.5), Profit = 21')

# Objective function contours (iso-profit lines)
for profit_level in [5, 10, 15, 21]:
    y_obj = (profit_level - 5*x) / 4
    ax.plot(x, y_obj, '--', color='red', alpha=0.3, linewidth=1)
    # Label the iso-profit line
    idx = np.argmin(np.abs(y_obj - 0.5))
    if 0 < x[idx] < 4.5:
        ax.text(x[idx], y_obj[idx] + 0.1, f'Profit={profit_level}',
                color='red', fontsize=8, alpha=0.6)

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 5)
ax.set_xlabel('x₁ (Product 1 quantity)', fontsize=12)
ax.set_ylabel('x₂ (Product 2 quantity)', fontsize=12)
ax.set_title('Linear Programming: Graphical Solution', fontsize=14)
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The optimal solution is at vertex (3, 1.5) with profit = 21.")
print("Notice: the optimum is at a vertex of the feasible polytope — this always")
print("happens for linear programs.")

In [ ]:
# Solving the same LP with scipy.optimize.linprog
# Note: linprog MINIMIZES, so we negate the objective for maximization

# Maximize 5*x1 + 4*x2 → Minimize -5*x1 - 4*x2
c = [-5, -4]  # negated for minimization

# Inequality constraints: A_ub @ x <= b_ub
A_ub = [
    [6, 4],   # 6x1 + 4x2 <= 24
    [1, 2],   # x1 + 2x2 <= 6
]
b_ub = [24, 6]

# Bounds: x1 >= 0, x2 >= 0
x1_bounds = (0, None)
x2_bounds = (0, None)

result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=[x1_bounds, x2_bounds], method='highs')

print("=== LP Solution (scipy.optimize.linprog) ===")
print(f"Optimal x₁: {result.x[0]:.4f}")
print(f"Optimal x₂: {result.x[1]:.4f}")
print(f"Maximum profit: {-result.fun:.4f}")  # negate back
print(f"Solver status: {result.message}")

### Deep Dive: Why LPs Always Have Vertex Solutions

A linear objective function $\mathbf{c}^T \mathbf{x}$ defines a family of parallel hyperplanes (lines in 2D). As we slide these hyperplanes in the direction of improvement, the last point of contact with the feasible polytope must be a vertex (or an edge/face, in degenerate cases where the objective is parallel to a constraint).

This geometric fact is the foundation of the Simplex method: instead of searching the entire feasible region, we only need to check vertices.

#### Common Misconceptions

| Misconception | Reality |
|---------------|---------|
| LP solutions can be interior points | The optimum is always on the boundary (at a vertex or edge) |
| More constraints make the problem harder | More constraints reduce the feasible region, often making the search easier |
| LP can only handle simple problems | LP underlies many sophisticated algorithms (network flows, assignment problems) |

---

## 3. The Simplex Method

### Intuitive Explanation

The **Simplex method** (George Dantzig, 1947) is the classic algorithm for solving linear programs. Its strategy is elegantly simple:

1. **Start** at any vertex of the feasible polytope
2. **Look** at all neighboring vertices (connected by an edge)
3. **Move** to the neighbor that improves the objective the most
4. **Repeat** until no neighbor is better — you are at the optimum

**What this means:** Imagine standing on one corner of a diamond. You look along each edge and walk to whichever adjacent corner is downhill. When every direction is uphill, you have found the lowest corner.

In theory, the Simplex method could visit exponentially many vertices. In practice, it is remarkably fast — typically visiting $O(m)$ vertices where $m$ is the number of constraints.

In [ ]:
# Visualize the Simplex method: vertex hopping
# We'll trace the path the Simplex method takes on our factory LP

fig, ax = plt.subplots(figsize=(10, 8))

x = np.linspace(0, 5, 400)
y1 = (24 - 6*x) / 4
y2 = (6 - x) / 2

# Feasible region
vertices = np.array([[0, 0], [4, 0], [3, 1.5], [0, 3]])
feasible = Polygon(vertices, alpha=0.15, color='green')
ax.add_patch(feasible)
ax.plot(x, np.clip(y1, 0, 10), 'b-', linewidth=1.5, alpha=0.5)
ax.plot(x, np.clip(y2, 0, 10), color='orange', linewidth=1.5, alpha=0.5)

# Simplex path: start at origin, hop along vertices
simplex_path = [
    [0, 0],     # Step 0: start at origin
    [4, 0],     # Step 1: move along x1 axis (improves objective most)
    [3, 1.5],   # Step 2: move to intersection vertex (still improving)
]

colors_path = ['blue', 'purple', 'red']
labels = ['Start: (0,0)\nProfit=0', 'Step 1: (4,0)\nProfit=20', 'Optimal: (3,1.5)\nProfit=21']

for i, (point, color, label) in enumerate(zip(simplex_path, colors_path, labels)):
    ax.plot(point[0], point[1], 'o', color=color, markersize=12, zorder=5)
    offset = [(0.2, 0.3), (0.2, 0.3), (-2.0, 0.3)][i]
    ax.annotate(label, xy=point, xytext=(point[0]+offset[0], point[1]+offset[1]),
                fontsize=10, fontweight='bold', color=color,
                arrowprops=dict(arrowstyle='->', color=color, lw=1.5))

# Draw path arrows
for i in range(len(simplex_path) - 1):
    start = simplex_path[i]
    end = simplex_path[i+1]
    ax.annotate('', xy=end, xytext=start,
                arrowprops=dict(arrowstyle='->', color='darkgreen', lw=2.5,
                                connectionstyle='arc3,rad=0.1'))

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 4.5)
ax.set_xlabel('x₁', fontsize=12)
ax.set_ylabel('x₂', fontsize=12)
ax.set_title('Simplex Method: Vertex Hopping', fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The Simplex method visits only 3 of 4 vertices to find the optimum.")
print("It never explores the interior — only vertices and edges.")

In [ ]:
def simplex_2d(c, A_ub, b_ub):
    """
    A simplified 2D Simplex implementation for educational purposes.
    Finds all vertices of the feasible polytope and evaluates the objective.
    
    This is NOT a full Simplex implementation — it is a brute-force vertex
    enumeration to illustrate the concept.
    
    Args:
        c: Objective coefficients [c1, c2] (minimize c^T x)
        A_ub: Inequality constraint matrix (A_ub @ x <= b_ub)
        b_ub: Right-hand side of inequalities
    
    Returns:
        Tuple of (optimal_vertex, optimal_value, all_vertices)
    """
    from itertools import combinations
    
    c = np.array(c)
    A = np.array(A_ub)
    b = np.array(b_ub)
    
    # Add non-negativity constraints: -x1 <= 0, -x2 <= 0
    A_full = np.vstack([A, [[-1, 0], [0, -1]]])
    b_full = np.concatenate([b, [0, 0]])
    
    n_constraints = len(A_full)
    vertices = []
    
    # Find all intersections of pairs of constraint lines
    for i, j in combinations(range(n_constraints), 2):
        A_pair = A_full[[i, j]]
        b_pair = b_full[[i, j]]
        try:
            vertex = np.linalg.solve(A_pair, b_pair)
            # Check if this vertex satisfies ALL constraints
            if np.all(A_full @ vertex <= b_full + 1e-10):
                vertices.append(vertex)
        except np.linalg.LinAlgError:
            continue  # Parallel constraints, no intersection
    
    if not vertices:
        return None, None, []
    
    vertices = np.array(vertices)
    # Remove duplicates
    unique = [vertices[0]]
    for v in vertices[1:]:
        if not any(np.allclose(v, u) for u in unique):
            unique.append(v)
    vertices = np.array(unique)
    
    # Evaluate objective at each vertex
    obj_values = vertices @ c
    best_idx = np.argmin(obj_values)
    
    return vertices[best_idx], obj_values[best_idx], vertices


# Test on our factory problem (minimize -5x1 - 4x2)
opt_vertex, opt_value, all_vertices = simplex_2d(
    c=[-5, -4],
    A_ub=[[6, 4], [1, 2]],
    b_ub=[24, 6]
)

print("=== Simplified Simplex (Vertex Enumeration) ===")
print(f"\nAll feasible vertices:")
for v in all_vertices:
    profit = 5*v[0] + 4*v[1]
    print(f"  ({v[0]:.2f}, {v[1]:.2f}) → Profit = {profit:.2f}")
print(f"\nOptimal vertex: ({opt_vertex[0]:.2f}, {opt_vertex[1]:.2f})")
print(f"Maximum profit: {-opt_value:.2f}")

### Deep Dive: Simplex vs Interior Point Methods

The Simplex method walks along edges of the polytope. An alternative family of algorithms — **interior point methods** — takes a very different approach: they walk *through the interior* of the polytope, approaching the optimal vertex from inside.

| Property | Simplex | Interior Point |
|----------|---------|----------------|
| Path | Along edges (vertex to vertex) | Through the interior |
| Worst case | Exponential | Polynomial |
| Practical speed | Very fast for most LPs | Slightly slower on small LPs, faster on huge ones |
| Solution type | Exact vertex solution | Approaches vertex asymptotically |
| Used in `scipy` | `method='revised simplex'` | `method='highs'` (default, hybrid) |

**What this means for ML:** Most modern LP/QP solvers (like those inside SVM implementations) use interior point methods because they have reliable polynomial-time guarantees.

---

## 4. Duality Theory

### Intuitive Explanation

Every linear program (the **primal**) has a companion problem called the **dual**. If the primal asks "What is the minimum cost to meet these requirements?", the dual asks "What is the maximum value of the resources?"

**Primal (minimize):**
$$\min_{x} \mathbf{c}^T \mathbf{x} \quad \text{s.t.} \quad A\mathbf{x} \geq \mathbf{b}, \; \mathbf{x} \geq 0$$

**Dual (maximize):**
$$\max_{y} \mathbf{b}^T \mathbf{y} \quad \text{s.t.} \quad A^T\mathbf{y} \leq \mathbf{c}, \; \mathbf{y} \geq 0$$

The dual variables $\mathbf{y}$ are sometimes called **shadow prices** — they tell you how much the optimal objective would change if you relaxed a constraint by one unit.

### Key Duality Theorems

1. **Weak Duality:** The dual objective is always a lower bound for the primal:
   $$\mathbf{b}^T \mathbf{y} \leq \mathbf{c}^T \mathbf{x} \quad \text{(for all feasible } x, y\text{)}$$

2. **Strong Duality:** At optimality, the bounds meet exactly:
   $$\mathbf{b}^T \mathbf{y}^* = \mathbf{c}^T \mathbf{x}^*$$

3. **Complementary Slackness:** At optimality, either a constraint is tight (active) or its dual variable is zero. No resource is both unused and valued.

**What this means:** Duality gives us two views of the same problem. If we can solve the dual, we automatically get the primal solution (and vice versa). In ML, the SVM dual formulation is actually easier to solve than the primal.

In [ ]:
# Demonstrate duality with a concrete example
# Primal: min c^T x  s.t. Ax >= b, x >= 0
# Reformulated as: min c^T x  s.t. -Ax <= -b, x >= 0 (for linprog)

# Primal problem:
# min 4x1 + 3x2
# s.t.  2x1 + x2 >= 8
#        x1 + 2x2 >= 6
#        x1, x2 >= 0

# Primal with linprog (convert >= to <=)
c_primal = [4, 3]
A_primal = [[-2, -1], [-1, -2]]  # negated for <=
b_primal = [-8, -6]

result_primal = linprog(c_primal, A_ub=A_primal, b_ub=b_primal,
                        bounds=[(0, None), (0, None)], method='highs')

# Dual problem:
# max 8y1 + 6y2 → min -8y1 - 6y2
# s.t.  2y1 + y2 <= 4
#        y1 + 2y2 <= 3
#        y1, y2 >= 0

c_dual = [-8, -6]  # negated for minimization
A_dual = [[2, 1], [1, 2]]
b_dual = [4, 3]

result_dual = linprog(c_dual, A_ub=A_dual, b_ub=b_dual,
                      bounds=[(0, None), (0, None)], method='highs')

print("=== Primal-Dual Comparison ===")
print(f"\nPrimal optimal x: ({result_primal.x[0]:.4f}, {result_primal.x[1]:.4f})")
print(f"Primal optimal value: {result_primal.fun:.4f}")
print(f"\nDual optimal y: ({result_dual.x[0]:.4f}, {result_dual.x[1]:.4f})")
print(f"Dual optimal value: {-result_dual.fun:.4f}")  # negate back
print(f"\nStrong duality holds: {np.isclose(result_primal.fun, -result_dual.fun)}")
print(f"Duality gap: {abs(result_primal.fun - (-result_dual.fun)):.10f}")

In [ ]:
# Visualize primal and dual feasible regions side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

x = np.linspace(0, 6, 400)

# --- Primal ---
ax = axes[0]
# Constraints: 2x1 + x2 >= 8, x1 + 2x2 >= 6
y_c1 = 8 - 2*x       # x2 >= 8 - 2x1
y_c2 = (6 - x) / 2   # x2 >= (6 - x1)/2

ax.plot(x, y_c1, 'b-', linewidth=2, label='2x₁ + x₂ = 8')
ax.plot(x, y_c2, color='orange', linewidth=2, label='x₁ + 2x₂ = 6')

# Fill feasible region (above both lines)
y_lower = np.maximum(y_c1, y_c2)
y_lower = np.maximum(y_lower, 0)
ax.fill_between(x, y_lower, 10, where=(y_lower <= 10) & (x >= 0),
                alpha=0.15, color='green', label='Feasible region')

# Optimal point
ax.plot(result_primal.x[0], result_primal.x[1], 'r*', markersize=15,
        label=f'Optimal: ({result_primal.x[0]:.1f}, {result_primal.x[1]:.1f})', zorder=5)

ax.set_xlim(-0.5, 6)
ax.set_ylim(-0.5, 10)
ax.set_xlabel('x₁', fontsize=12)
ax.set_ylabel('x₂', fontsize=12)
ax.set_title(f'Primal Problem (min = {result_primal.fun:.2f})', fontsize=13)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# --- Dual ---
ax = axes[1]
# Constraints: 2y1 + y2 <= 4, y1 + 2y2 <= 3
y_d1 = 4 - 2*x       # y2 <= 4 - 2y1
y_d2 = (3 - x) / 2   # y2 <= (3 - y1)/2

ax.plot(x, y_d1, 'b-', linewidth=2, label='2y₁ + y₂ = 4')
ax.plot(x, y_d2, color='orange', linewidth=2, label='y₁ + 2y₂ = 3')

# Feasible region vertices for dual
dual_verts = np.array([[0, 0], [2, 0], [5/3, 2/3], [0, 1.5]])
dual_poly = Polygon(dual_verts, alpha=0.15, color='green', label='Feasible region')
ax.add_patch(dual_poly)

# Optimal point
ax.plot(result_dual.x[0], result_dual.x[1], 'r*', markersize=15,
        label=f'Optimal: ({result_dual.x[0]:.2f}, {result_dual.x[1]:.2f})', zorder=5)

ax.set_xlim(-0.5, 3.5)
ax.set_ylim(-0.5, 3)
ax.set_xlabel('y₁', fontsize=12)
ax.set_ylabel('y₂', fontsize=12)
ax.set_title(f'Dual Problem (max = {-result_dual.fun:.2f})', fontsize=13)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Strong duality: both problems achieve the same optimal value.")
print("The dual variables tell us the 'shadow prices' of the primal constraints.")

### Deep Dive: Why Duality Matters in ML

Duality is not just a theoretical curiosity — it is the engine behind several important ML algorithms:

| ML Application | How Duality is Used |
|----------------|---------------------|
| **Support Vector Machines** | The SVM primal is hard (minimize over $w, b$). The dual is easier and reveals the kernel trick. |
| **Regularization** | Adding an $\ell_2$ penalty is equivalent to constraining the norm (Lagrangian duality). |
| **Maximum Entropy Models** | The dual of the constrained entropy problem yields logistic regression. |
| **Variational Inference** | ELBO (Evidence Lower BOund) is a dual bound on the marginal likelihood. |

#### Key Insight

The dual often reveals hidden structure. For SVMs, the dual formulation depends only on dot products of data points $\langle x_i, x_j \rangle$, which lets us use the **kernel trick** to work in infinite-dimensional feature spaces.

---

## 5. Convex Optimization

### Intuitive Explanation

A set is **convex** if the line segment between any two points in the set lies entirely within the set. Think of a filled circle (convex) versus a crescent moon (not convex).

A function is **convex** if it curves upward — like a bowl. Formally, $f$ is convex if:

$$f(\lambda x + (1-\lambda)y) \leq \lambda f(x) + (1-\lambda)f(y) \quad \forall \lambda \in [0,1]$$

**What this means:** Draw a line between any two points on the function's graph. If the function always lies below (or on) the line, it is convex. A bowl has this property; a roller coaster does not.

### Why Convexity Matters

**The fundamental theorem of convex optimization:** For a convex function over a convex set, every local minimum is a global minimum.

This means:
- No getting stuck in local minima
- Gradient descent is guaranteed to find the global optimum
- The solution is unique (for strictly convex functions)

This is why much of ML theory revolves around convex loss functions (squared error, cross-entropy, hinge loss) and convex regularizers ($\ell_1$, $\ell_2$).

In [ ]:
# Visualize convex vs non-convex sets and functions
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# --- Top Left: Convex set ---
ax = axes[0, 0]
theta = np.linspace(0, 2*np.pi, 100)
# Ellipse (convex)
cx, cy = 2*np.cos(theta), np.sin(theta)
ax.fill(cx, cy, alpha=0.3, color='green')
ax.plot(cx, cy, 'g-', linewidth=2)
# Show line segment test
p1, p2 = np.array([-1.5, 0.3]), np.array([1.2, -0.6])
ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'r-', linewidth=2, label='Line segment (inside)')
ax.plot(*p1, 'ro', markersize=8)
ax.plot(*p2, 'ro', markersize=8)
ax.set_title('Convex Set', fontsize=13)
ax.set_aspect('equal')
ax.legend()
ax.grid(True, alpha=0.3)

# --- Top Right: Non-convex set ---
ax = axes[0, 1]
# Star shape (non-convex)
angles = np.linspace(0, 2*np.pi, 11)
radii = [1.5 if i % 2 == 0 else 0.6 for i in range(11)]
sx = [r*np.cos(a) for r, a in zip(radii, angles)]
sy = [r*np.sin(a) for r, a in zip(radii, angles)]
ax.fill(sx, sy, alpha=0.3, color='red')
ax.plot(sx, sy, 'r-', linewidth=2)
# Show line segment test failing
p1, p2 = np.array([-1.0, 0.8]), np.array([1.0, 0.8])
ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'b-', linewidth=2, label='Line segment (goes outside!)')
ax.plot(*p1, 'bo', markersize=8)
ax.plot(*p2, 'bo', markersize=8)
ax.set_title('Non-Convex Set', fontsize=13)
ax.set_aspect('equal')
ax.legend()
ax.grid(True, alpha=0.3)

# --- Bottom Left: Convex function ---
ax = axes[1, 0]
x = np.linspace(-3, 3, 200)
y_convex = x**2
ax.plot(x, y_convex, 'g-', linewidth=2.5, label='f(x) = x² (convex)')
# Secant line
xa, xb = -2, 1.5
ya, yb = xa**2, xb**2
ax.plot([xa, xb], [ya, yb], 'r--', linewidth=2, label='Secant line (always above f)')
ax.fill_between(x[(x >= xa) & (x <= xb)],
                x[(x >= xa) & (x <= xb)]**2,
                np.interp(x[(x >= xa) & (x <= xb)], [xa, xb], [ya, yb]),
                alpha=0.2, color='green')
ax.plot(0, 0, 'r*', markersize=15, label='Global minimum', zorder=5)
ax.set_title('Convex Function', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# --- Bottom Right: Non-convex function ---
ax = axes[1, 1]
y_nonconvex = np.sin(2*x) + 0.3*x**2
ax.plot(x, y_nonconvex, 'r-', linewidth=2.5, label='f(x) = sin(2x) + 0.3x² (non-convex)')
# Mark local minima
from scipy.signal import argrelmin
local_mins = argrelmin(y_nonconvex, order=20)[0]
for lm in local_mins:
    color = 'red' if y_nonconvex[lm] == min(y_nonconvex[local_mins]) else 'orange'
    label = 'Global minimum' if color == 'red' else 'Local minimum'
    ax.plot(x[lm], y_nonconvex[lm], '*', color=color, markersize=15, zorder=5, label=label)
ax.set_title('Non-Convex Function', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Top row: Convex sets pass the line-segment test; non-convex sets fail it.")
print("Bottom row: Convex functions have one minimum; non-convex functions may have many.")

In [ ]:
# Demonstrate: gradient descent finds the global optimum on convex functions
# but can get stuck on non-convex functions

def gradient_descent_1d(f, df, x0, lr=0.1, n_steps=30):
    """
    Simple 1D gradient descent.
    
    Args:
        f: Objective function
        df: Gradient of objective
        x0: Starting point
        lr: Learning rate
        n_steps: Number of steps
    
    Returns:
        List of (x, f(x)) at each step
    """
    path = [(x0, f(x0))]
    x = x0
    for _ in range(n_steps):
        x = x - lr * df(x)
        path.append((x, f(x)))
    return path


# Convex: f(x) = (x-1)^2
f_convex = lambda x: (x - 1)**2
df_convex = lambda x: 2*(x - 1)

# Non-convex: f(x) = sin(3x) + 0.3*x^2
f_nonconvex = lambda x: np.sin(3*x) + 0.3*x**2
df_nonconvex = lambda x: 3*np.cos(3*x) + 0.6*x

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x_range = np.linspace(-3, 4, 300)

# Convex case
ax = axes[0]
ax.plot(x_range, f_convex(x_range), 'g-', linewidth=2, label='f(x) = (x-1)²')
for x0, color, ls in [(-2.5, 'blue', '-'), (3.5, 'red', '--')]:
    path = gradient_descent_1d(f_convex, df_convex, x0, lr=0.1, n_steps=20)
    xs, ys = zip(*path)
    ax.plot(xs, ys, 'o-', color=color, markersize=4, linewidth=1, alpha=0.7,
            label=f'Start x={x0}')
    ax.plot(xs[0], ys[0], 's', color=color, markersize=10)
ax.set_title('Convex: Both paths find global min', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.legend()
ax.grid(True, alpha=0.3)

# Non-convex case
ax = axes[1]
ax.plot(x_range, f_nonconvex(x_range), 'r-', linewidth=2, label='f(x) = sin(3x) + 0.3x²')
for x0, color, ls in [(-2.0, 'blue', '-'), (2.0, 'red', '--')]:
    path = gradient_descent_1d(f_nonconvex, df_nonconvex, x0, lr=0.05, n_steps=40)
    xs, ys = zip(*path)
    ax.plot(xs, ys, 'o-', color=color, markersize=4, linewidth=1, alpha=0.7,
            label=f'Start x={x0}')
    ax.plot(xs[0], ys[0], 's', color=color, markersize=10)
ax.set_title('Non-convex: Different starts → different minima', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Convex functions: Every starting point leads to the same global minimum.")
print("Non-convex functions: Different starting points can lead to different local minima.")
print("\nThis is why neural network training (non-convex) depends on initialization,")
print("while logistic regression (convex) converges to the same solution regardless.")

### Deep Dive: Common Convex Functions in ML

Many loss functions used in ML are convex by design:

| Function | Formula | Convex? | Used in |
|----------|---------|---------|----------|
| **Squared error** | $(y - \hat{y})^2$ | Yes (strictly) | Linear regression |
| **Cross-entropy** | $-y \log \hat{y} - (1-y)\log(1-\hat{y})$ | Yes (strictly) | Logistic regression |
| **Hinge loss** | $\max(0, 1 - y \hat{y})$ | Yes (not strictly) | SVM |
| **$\ell_2$ penalty** | $\|\mathbf{w}\|_2^2$ | Yes (strictly) | Ridge regression |
| **$\ell_1$ penalty** | $\|\mathbf{w}\|_1$ | Yes (not strictly) | LASSO |
| **Neural network loss** | Complex composition | No (generally) | Deep learning |

#### Key Insight

A sum of convex functions is convex. This is why "loss + regularization" stays convex when both components are convex: the composite objective inherits the nice optimization properties of its parts.

In [ ]:
# Visualize common ML loss functions and their convexity
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

z = np.linspace(-3, 3, 300)

# Squared loss
ax = axes[0]
ax.plot(z, z**2, 'b-', linewidth=2.5)
ax.set_title('Squared Loss: (y - ŷ)²', fontsize=12)
ax.set_xlabel('y - ŷ (residual)')
ax.set_ylabel('Loss')
ax.text(0.05, 0.85, 'Convex ✓\nSmooth ✓\nStrictly convex ✓',
        transform=ax.transAxes, fontsize=10, color='green',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax.grid(True, alpha=0.3)

# Hinge loss
ax = axes[1]
hinge = np.maximum(0, 1 - z)
ax.plot(z, hinge, 'r-', linewidth=2.5)
ax.set_title('Hinge Loss: max(0, 1 - yŷ)', fontsize=12)
ax.set_xlabel('y·ŷ (margin)')
ax.set_ylabel('Loss')
ax.text(0.05, 0.85, 'Convex ✓\nNot smooth ✗\nNot strictly convex ✗',
        transform=ax.transAxes, fontsize=10, color='orange',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax.grid(True, alpha=0.3)

# Cross-entropy / logistic loss
ax = axes[2]
logistic_loss = np.log(1 + np.exp(-z))
ax.plot(z, logistic_loss, 'purple', linewidth=2.5)
ax.set_title('Logistic Loss: log(1 + e⁻ᶻ)', fontsize=12)
ax.set_xlabel('y·ŷ (margin)')
ax.set_ylabel('Loss')
ax.text(0.05, 0.85, 'Convex ✓\nSmooth ✓\nStrictly convex ✓',
        transform=ax.transAxes, fontsize=10, color='green',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("All three are convex — gradient descent will find the global minimum.")
print("Squared loss is smooth and easy to optimize.")
print("Hinge loss has a 'kink' at 1, requiring subgradient methods.")
print("Logistic loss is a smooth approximation of hinge loss.")

---

## 6. Constrained Optimization & Lagrange Multipliers

### Intuitive Explanation

How do you minimize a function when you must stay on a specific curve or surface? This is the **constrained optimization** problem.

**The setup:**
$$\min_{x} f(x) \quad \text{subject to} \quad g(x) = 0$$

**Lagrange's insight (1788):** At the constrained optimum, the gradient of the objective $\nabla f$ must be parallel to the gradient of the constraint $\nabla g$. If they were not parallel, you could slide along the constraint and still improve the objective.

This gives us the **Lagrangian**:
$$\mathcal{L}(x, \lambda) = f(x) + \lambda \, g(x)$$

The optimal point satisfies:
$$\nabla_x \mathcal{L} = 0 \quad \text{and} \quad \nabla_\lambda \mathcal{L} = 0$$

The variable $\lambda$ is called the **Lagrange multiplier**. It tells you the rate at which the optimal objective value changes as you relax the constraint.

**What this means:** Imagine hiking downhill but you must stay on a trail (the constraint). You stop when the steepest downhill direction is perpendicular to the trail — because moving along the trail no longer takes you downhill.

In [ ]:
# Geometric visualization of Lagrange multipliers
# Minimize f(x,y) = x^2 + y^2 subject to x + y = 4

fig, ax = plt.subplots(figsize=(10, 8))

x_range = np.linspace(-1, 6, 300)
y_range = np.linspace(-1, 6, 300)
X, Y = np.meshgrid(x_range, y_range)
Z = X**2 + Y**2  # objective

# Contours of the objective
levels = [1, 2, 4, 8, 12, 16, 20, 25, 32]
contour = ax.contour(X, Y, Z, levels=levels, cmap='Blues', alpha=0.7)
ax.clabel(contour, fmt='%.0f', fontsize=9)

# Constraint: x + y = 4
ax.plot(x_range, 4 - x_range, 'r-', linewidth=3, label='Constraint: x + y = 4')

# Optimal point: x = y = 2 (by symmetry, or solve the system)
opt_x, opt_y = 2, 2
ax.plot(opt_x, opt_y, 'r*', markersize=20, zorder=5, label='Constrained optimum (2, 2)')

# Draw gradients at the optimal point
# grad f = (2x, 2y) = (4, 4)
# grad g = (1, 1)  where g(x,y) = x + y - 4
scale = 0.3
ax.annotate('', xy=(opt_x + 4*scale, opt_y + 4*scale), xytext=(opt_x, opt_y),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2.5))
ax.text(opt_x + 4*scale + 0.1, opt_y + 4*scale + 0.1, '∇f = (4, 4)',
        fontsize=11, color='blue', fontweight='bold')

ax.annotate('', xy=(opt_x + 1*scale*2, opt_y + 1*scale*2), xytext=(opt_x, opt_y),
            arrowprops=dict(arrowstyle='->', color='green', lw=2.5))
ax.text(opt_x + 1*scale*2 - 0.7, opt_y + 1*scale*2 + 0.2, '∇g = (1, 1)',
        fontsize=11, color='green', fontweight='bold')

# Unconstrained optimum
ax.plot(0, 0, 'b*', markersize=15, alpha=0.4, label='Unconstrained optimum (0, 0)')

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Lagrange Multipliers: Gradient Alignment at Optimum', fontsize=14)
ax.legend(fontsize=10, loc='upper right')
ax.set_aspect('equal')
ax.set_xlim(-1, 6)
ax.set_ylim(-1, 6)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("At the constrained optimum (2, 2):")
print("  ∇f = (4, 4) is parallel to ∇g = (1, 1)")
print("  λ = 4 (the Lagrange multiplier: ∇f = λ·∇g)")
print("  The multiplier λ = 4 means: relaxing the constraint by ε")
print("  would improve the objective by approximately 4ε.")

In [ ]:
# Solving constrained optimization with scipy.optimize.minimize

# Problem: minimize x^2 + y^2 subject to x + y = 4
def objective(xy):
    return xy[0]**2 + xy[1]**2

def constraint_eq(xy):
    return xy[0] + xy[1] - 4  # equals zero at feasibility

# Using SLSQP (Sequential Least Squares Quadratic Programming)
result = minimize(
    objective,
    x0=[0, 0],  # starting guess
    method='SLSQP',
    constraints={'type': 'eq', 'fun': constraint_eq}
)

print("=== Constrained Optimization with scipy ===")
print(f"Optimal point: ({result.x[0]:.4f}, {result.x[1]:.4f})")
print(f"Optimal value: {result.fun:.4f}")
print(f"Constraint satisfied: {np.isclose(constraint_eq(result.x), 0)}")
print(f"Solver converged: {result.success}")

# Verify: analytical solution is (2, 2) with f = 8
print(f"\nAnalytical solution: (2, 2), f = 8")
print(f"Match: {np.allclose(result.x, [2, 2]) and np.isclose(result.fun, 8)}")

### KKT Conditions: The Full Picture

Lagrange multipliers handle **equality** constraints. For **inequality** constraints ($g_i(x) \leq 0$), we need the **Karush-Kuhn-Tucker (KKT) conditions**, which generalize Lagrange multipliers.

For the problem:
$$\min_x f(x) \quad \text{s.t.} \quad g_i(x) \leq 0, \quad h_j(x) = 0$$

The KKT conditions are:

| Condition | Statement | Intuition |
|-----------|-----------|----------|
| **Stationarity** | $\nabla f + \sum \mu_i \nabla g_i + \sum \lambda_j \nabla h_j = 0$ | Gradient balance at optimum |
| **Primal feasibility** | $g_i(x) \leq 0, \; h_j(x) = 0$ | Solution must be feasible |
| **Dual feasibility** | $\mu_i \geq 0$ | Inequality multipliers are non-negative |
| **Complementary slackness** | $\mu_i \, g_i(x) = 0$ | Either constraint is active or multiplier is zero |

**What this means:** 
- If a constraint is **not active** ($g_i(x) < 0$, you are strictly inside the boundary), then $\mu_i = 0$ — the constraint does not affect the solution.
- If a constraint **is active** ($g_i(x) = 0$, you are on the boundary), then $\mu_i > 0$ — the constraint is "pushing back" and preventing you from going further.

For **convex** problems, the KKT conditions are both necessary and sufficient for optimality.

In [ ]:
# Visualize KKT conditions: inequality constraints
# Minimize f(x,y) = (x-3)^2 + (y-3)^2 subject to x + y <= 4, x >= 0, y >= 0

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

x_range = np.linspace(-0.5, 5, 300)
y_range = np.linspace(-0.5, 5, 300)
X, Y = np.meshgrid(x_range, y_range)

# Case 1: Unconstrained optimum is feasible
ax = axes[0]
Z = (X - 1.5)**2 + (Y - 1)**2
contour = ax.contour(X, Y, Z, levels=10, cmap='Blues', alpha=0.7)
ax.clabel(contour, fmt='%.1f', fontsize=8)

# Feasible region: x + y <= 4, x >= 0, y >= 0
feas_verts = np.array([[0, 0], [4, 0], [0, 4]])
feas_poly = Polygon(feas_verts, alpha=0.15, color='green', label='Feasible region')
ax.add_patch(feas_poly)
ax.plot([0, 4], [4, 0], 'g-', linewidth=2)

ax.plot(1.5, 1, 'r*', markersize=15, zorder=5, label='Optimum (1.5, 1) — interior')
ax.set_title('Constraint NOT Active (μ = 0)', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend(fontsize=9)
ax.set_aspect('equal')
ax.set_xlim(-0.5, 5)
ax.set_ylim(-0.5, 5)
ax.grid(True, alpha=0.3)

# Case 2: Unconstrained optimum is infeasible → constraint active
ax = axes[1]
Z2 = (X - 3)**2 + (Y - 3)**2
contour = ax.contour(X, Y, Z2, levels=10, cmap='Blues', alpha=0.7)
ax.clabel(contour, fmt='%.1f', fontsize=8)

feas_poly2 = Polygon(feas_verts, alpha=0.15, color='green', label='Feasible region')
ax.add_patch(feas_poly2)
ax.plot([0, 4], [4, 0], 'g-', linewidth=2)

# Constrained optimum: closest point on x+y=4 to (3,3) is (2,2)
ax.plot(3, 3, 'b*', markersize=12, alpha=0.4, label='Unconstrained optimum (3, 3)')
ax.plot(2, 2, 'r*', markersize=15, zorder=5, label='Constrained optimum (2, 2) — on boundary')
ax.annotate('', xy=(2, 2), xytext=(3, 3),
            arrowprops=dict(arrowstyle='->', color='red', lw=1.5, ls='--'))

ax.set_title('Constraint IS Active (μ > 0)', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend(fontsize=9)
ax.set_aspect('equal')
ax.set_xlim(-0.5, 5)
ax.set_ylim(-0.5, 5)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Left: The unconstrained optimum (1.5, 1) is inside the feasible region.")
print("  → The constraint is NOT active, and the multiplier μ = 0.")
print("  → The constraint doesn't affect the solution at all.")
print("\nRight: The unconstrained optimum (3, 3) is OUTSIDE the feasible region.")
print("  → The constraint IS active, pushing the solution to (2, 2) on the boundary.")
print("  → The multiplier μ > 0, representing the 'cost' of the constraint.")

### Deep Dive: Complementary Slackness — The Either/Or Rule

Complementary slackness ($\mu_i \cdot g_i(x) = 0$) is perhaps the most elegant condition. It says:

For each inequality constraint, exactly one of two things is true:
1. The constraint is **slack** ($g_i(x) < 0$): the boundary is not reached, so the multiplier is zero ($\mu_i = 0$).
2. The constraint is **tight** ($g_i(x) = 0$): the boundary is hit, and the multiplier is positive ($\mu_i > 0$).

In ML terms:
- In an SVM, most data points have $\mu_i = 0$ — they do not affect the decision boundary.
- Only the **support vectors** (points on the margin boundary) have $\mu_i > 0$.
- This is why SVMs are sparse: the solution depends only on a few critical data points.

---

## 7. Connection to Machine Learning

### ML as Optimization

Nearly every ML algorithm can be framed as an optimization problem:

| Algorithm | Objective | Constraints | Type |
|-----------|-----------|-------------|------|
| **Linear regression** | $\min_w \|Xw - y\|^2$ | None | Unconstrained convex |
| **Ridge regression** | $\min_w \|Xw - y\|^2 + \alpha\|w\|^2$ | None (equivalent to constrained) | Unconstrained convex |
| **LASSO** | $\min_w \|Xw - y\|^2 + \alpha\|w\|_1$ | None (equivalent to constrained) | Unconstrained convex |
| **Logistic regression** | $\min_w \sum \log(1 + e^{-y_i w^T x_i})$ | None | Unconstrained convex |
| **SVM (primal)** | $\min_{w,b} \frac{1}{2}\|w\|^2$ | $y_i(w^T x_i + b) \geq 1$ | Constrained convex (QP) |
| **SVM (dual)** | $\max_\alpha \sum \alpha_i - \frac{1}{2}\sum \alpha_i \alpha_j y_i y_j x_i^T x_j$ | $0 \leq \alpha_i \leq C$ | Constrained convex (QP) |
| **Neural networks** | $\min_\theta \mathcal{L}(f_\theta(X), y)$ | None | Unconstrained **non-convex** |

Notice: everything except neural networks is convex. This is why classical ML has nice convergence guarantees, while deep learning requires more art (learning rate schedules, initialization strategies, etc.).

In [ ]:
# Demonstrate: SVM as a constrained optimization problem
from sklearn.svm import SVC
from sklearn.datasets import make_blobs

# Generate linearly separable data
np.random.seed(42)
X_train, y_train = make_blobs(n_samples=40, centers=2, cluster_std=1.2, random_state=42)
y_train = 2*y_train - 1  # Convert to {-1, +1}

# Train SVM
svm = SVC(kernel='linear', C=10.0)
svm.fit(X_train, y_train)

# Extract the optimization solution
w = svm.coef_[0]
b = svm.intercept_[0]
support_vectors = svm.support_vectors_
n_support = len(support_vectors)

print("=== SVM as Optimization ===")
print(f"Weight vector w: [{w[0]:.4f}, {w[1]:.4f}]")
print(f"Bias b: {b:.4f}")
print(f"Margin width: {2 / np.linalg.norm(w):.4f}")
print(f"Number of support vectors: {n_support} out of {len(X_train)} points")
print(f"\nOnly {n_support}/{len(X_train)} points have non-zero dual variables (α > 0).")
print("This is complementary slackness in action!")

In [ ]:
# Visualize the SVM solution with margin and support vectors
fig, ax = plt.subplots(figsize=(10, 8))

# Plot data points
colors = ['blue' if y == -1 else 'red' for y in y_train]
for cls, color, label in [(-1, 'blue', 'Class -1'), (1, 'red', 'Class +1')]:
    mask = y_train == cls
    ax.scatter(X_train[mask, 0], X_train[mask, 1], c=color, s=50,
               edgecolors='k', linewidth=0.5, label=label, alpha=0.7)

# Plot support vectors
ax.scatter(support_vectors[:, 0], support_vectors[:, 1], s=200,
           facecolors='none', edgecolors='green', linewidths=2.5,
           label=f'Support vectors ({n_support})', zorder=5)

# Plot decision boundary and margins
x_boundary = np.linspace(X_train[:, 0].min() - 1, X_train[:, 0].max() + 1, 200)

# Decision boundary: w[0]*x + w[1]*y + b = 0
y_boundary = -(w[0] * x_boundary + b) / w[1]
y_margin_pos = -(w[0] * x_boundary + b - 1) / w[1]
y_margin_neg = -(w[0] * x_boundary + b + 1) / w[1]

ax.plot(x_boundary, y_boundary, 'k-', linewidth=2, label='Decision boundary')
ax.plot(x_boundary, y_margin_pos, 'k--', linewidth=1, alpha=0.5, label='Margin boundaries')
ax.plot(x_boundary, y_margin_neg, 'k--', linewidth=1, alpha=0.5)
ax.fill_between(x_boundary, y_margin_neg, y_margin_pos, alpha=0.08, color='green')

ax.set_xlabel('Feature 1', fontsize=12)
ax.set_ylabel('Feature 2', fontsize=12)
ax.set_title('SVM: Maximum-Margin Classifier (Constrained Optimization)', fontsize=14)
ax.legend(fontsize=10)
ax.set_xlim(X_train[:, 0].min() - 1, X_train[:, 0].max() + 1)
ax.set_ylim(X_train[:, 1].min() - 1, X_train[:, 1].max() + 1)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The SVM solves: min ½||w||² subject to yᵢ(w·xᵢ + b) ≥ 1")
print("This is a quadratic program (QP) — convex with linear constraints.")
print("The support vectors are the points with active constraints (on the margin).")

In [ ]:
# Demonstrate: Regularization as a constraint
# Ridge regression: min ||Xw - y||^2 + alpha * ||w||^2
# This is equivalent to: min ||Xw - y||^2 subject to ||w||^2 <= t

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Create a simple regression problem
np.random.seed(42)
w_true = np.array([3, -2])
X_reg = np.random.randn(20, 2)
y_reg = X_reg @ w_true + 0.5 * np.random.randn(20)

# Compute loss surface
w1_range = np.linspace(-1, 6, 200)
w2_range = np.linspace(-5, 2, 200)
W1, W2 = np.meshgrid(w1_range, w2_range)

Loss = np.zeros_like(W1)
for i in range(len(w1_range)):
    for j in range(len(w2_range)):
        w_test = np.array([W1[j, i], W2[j, i]])
        Loss[j, i] = np.sum((X_reg @ w_test - y_reg)**2) / len(y_reg)

# Left: Regularization view (penalized objective)
ax = axes[0]
ax.contour(W1, W2, Loss, levels=15, cmap='Blues', alpha=0.7)

# Show effect of different regularization strengths
for alpha, color, name in [(0, 'red', 'No reg (α=0)'), (2, 'orange', 'α=2'),
                            (10, 'green', 'α=10')]:
    # Analytical solution: (X^T X + alpha*I)^{-1} X^T y
    w_ridge = np.linalg.solve(X_reg.T @ X_reg + alpha * np.eye(2), X_reg.T @ y_reg)
    ax.plot(w_ridge[0], w_ridge[1], '*', color=color, markersize=15, label=name, zorder=5)

ax.set_xlabel('w₁', fontsize=12)
ax.set_ylabel('w₂', fontsize=12)
ax.set_title('Regularization Shrinks Weights Toward Origin', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Right: Constraint view (norm ball)
ax = axes[1]
ax.contour(W1, W2, Loss, levels=15, cmap='Blues', alpha=0.7)

# L2 constraint ball
theta = np.linspace(0, 2*np.pi, 100)
for radius, color, ls in [(2.0, 'green', '-'), (3.0, 'orange', '--'), (5.0, 'red', ':')]:
    ax.plot(radius*np.cos(theta), radius*np.sin(theta), color=color, linewidth=2,
            linestyle=ls, label=f'||w||₂ ≤ {radius}')

# OLS solution (no constraint)
w_ols = np.linalg.solve(X_reg.T @ X_reg, X_reg.T @ y_reg)
ax.plot(w_ols[0], w_ols[1], 'r*', markersize=15, label='OLS (unconstrained)', zorder=5)

ax.set_xlabel('w₁', fontsize=12)
ax.set_ylabel('w₂', fontsize=12)
ax.set_title('Equivalent View: Constraint on Weight Norm', fontsize=13)
ax.legend(fontsize=9)
ax.set_aspect('equal')
ax.set_xlim(-1, 6)
ax.set_ylim(-5, 2)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Left: Adding α||w||² to the loss shrinks weights toward zero.")
print("Right: Equivalent to constraining ||w||₂ ≤ t (Lagrangian duality).")
print("\nThis equivalence is Lagrangian duality in action:")
print("  Penalized form: min L(w) + α·R(w)")
print("  Constrained form: min L(w) s.t. R(w) ≤ t")
print("  The multiplier α and the constraint bound t are dual to each other.")

### Why This Matters in Machine Learning

| Optimization Concept | ML Application | How It's Used |
|----------------------|----------------|---------------|
| **Objective function** | Loss function | Measures how wrong the model is |
| **Constraints** | Regularization, fairness constraints | Limits model complexity or enforces properties |
| **Feasible region** | Weight space satisfying constraints | Set of allowed model parameters |
| **Convexity** | Guarantees on convergence | Convex losses guarantee finding the best model |
| **Lagrange multipliers** | SVM dual, regularization theory | Enable kernel trick, explain regularization |
| **KKT conditions** | Support vectors, active constraints | Identify which data points matter |
| **Duality** | SVM formulation, ELBO in VAEs | Provides alternative problem views |
| **Linear programming** | Network flow, assignment, scheduling | Combinatorial ML problems |

---

## Exercises

### Exercise 1: Formulate and Solve an LP

A bakery makes cakes and cookies.
- Each cake uses 3 cups of flour and 1 egg, and sells for \$8.
- Each batch of cookies uses 1 cup of flour and 2 eggs, and sells for \$5.
- The bakery has 12 cups of flour and 8 eggs available.

**Task:** Formulate this as an LP and solve it using `scipy.optimize.linprog`. How many cakes and cookie batches should the bakery make to maximize revenue?

In [ ]:
# EXERCISE 1: Solve the bakery LP
def solve_bakery_lp():
    """
    Formulate and solve the bakery LP.
    
    Returns:
        Tuple of (n_cakes, n_cookies, max_revenue)
    """
    # TODO: Implement this!
    # Hint: linprog MINIMIZES, so negate the objective for maximization.
    # Hint: c = [-8, -5] for the negated profit objective
    # Hint: A_ub represents [flour_constraint, egg_constraint]
    
    pass  # Replace with your solution

# Test
result = solve_bakery_lp()
if result is not None:
    n_cakes, n_cookies, max_revenue = result
    print(f"Cakes: {n_cakes:.2f}")
    print(f"Cookie batches: {n_cookies:.2f}")
    print(f"Maximum revenue: ${max_revenue:.2f}")
    
    # Expected: cakes=2.4, cookies=4.8/2=2.8... let's compute
    expected_revenue = 34.40
    print(f"\nExpected revenue: ${expected_revenue:.2f}")
    print(f"Correct: {np.isclose(max_revenue, expected_revenue, atol=0.1)}")

### Exercise 2: Verify Convexity

Write a function that numerically checks whether a 1D function is convex over a given interval by testing the secant line condition at random points.

In [ ]:
# EXERCISE 2: Check convexity numerically
def is_convex(f, x_min, x_max, n_tests=1000):
    """
    Numerically check if a function is convex over [x_min, x_max].
    
    A function is convex if for all x, y in the interval and lambda in [0,1]:
        f(lambda*x + (1-lambda)*y) <= lambda*f(x) + (1-lambda)*f(y)
    
    Args:
        f: Function to test
        x_min, x_max: Interval endpoints
        n_tests: Number of random tests
    
    Returns:
        True if all tests pass (function appears convex), False otherwise
    """
    # TODO: Implement this!
    # Hint: For each test, pick random x, y in [x_min, x_max]
    #       and random lambda in [0, 1], then check the inequality.
    
    pass  # Replace with your solution

# Test with known functions
f_quadratic = lambda x: x**2          # convex
f_absolute = lambda x: np.abs(x)       # convex
f_sine = lambda x: np.sin(x)           # NOT convex over [-pi, pi]
f_exp = lambda x: np.exp(x)            # convex

if is_convex is not None and is_convex(f_quadratic, -5, 5) is not None:
    print(f"x²: convex = {is_convex(f_quadratic, -5, 5)}")
    print(f"|x|: convex = {is_convex(f_absolute, -5, 5)}")
    print(f"sin(x) on [-π,π]: convex = {is_convex(f_sine, -np.pi, np.pi)}")
    print(f"eˣ: convex = {is_convex(f_exp, -5, 5)}")
    print(f"\nExpected: True, True, False, True")

### Exercise 3: Lagrange Multipliers by Hand (with Verification)

Solve this problem analytically, then verify with `scipy.optimize.minimize`:

$$\min_{x,y} \; x^2 + 2y^2 \quad \text{subject to} \quad x + y = 3$$

**Steps:**
1. Write the Lagrangian: $\mathcal{L} = x^2 + 2y^2 + \lambda(x + y - 3)$
2. Set partial derivatives to zero and solve
3. Verify numerically

In [ ]:
# EXERCISE 3: Lagrange multipliers
def solve_lagrange_problem():
    """
    Solve: min x^2 + 2y^2 subject to x + y = 3
    
    Analytical steps:
    L = x^2 + 2y^2 + lambda*(x + y - 3)
    dL/dx = 2x + lambda = 0      → x = -lambda/2
    dL/dy = 4y + lambda = 0      → y = -lambda/4
    dL/dlambda = x + y - 3 = 0   → -lambda/2 - lambda/4 = 3
                                  → -3*lambda/4 = 3
                                  → lambda = -4
    So: x = 2, y = 1, lambda = -4
    
    Returns:
        Tuple of (x_opt, y_opt, lambda_opt, f_opt)
    """
    # TODO: Return the analytical solution
    # Hint: Follow the steps in the docstring above
    
    pass  # Replace with your solution

# Numerical verification
result_num = minimize(
    lambda xy: xy[0]**2 + 2*xy[1]**2,
    x0=[0, 0],
    method='SLSQP',
    constraints={'type': 'eq', 'fun': lambda xy: xy[0] + xy[1] - 3}
)

print("=== Numerical Solution ===")
print(f"x = {result_num.x[0]:.4f}, y = {result_num.x[1]:.4f}")
print(f"f(x,y) = {result_num.fun:.4f}")

if solve_lagrange_problem is not None and solve_lagrange_problem() is not None:
    x_opt, y_opt, lambda_opt, f_opt = solve_lagrange_problem()
    print(f"\n=== Your Analytical Solution ===")
    print(f"x = {x_opt}, y = {y_opt}, λ = {lambda_opt}")
    print(f"f(x,y) = {f_opt}")
    print(f"\nMatch: {np.allclose([x_opt, y_opt], result_num.x, atol=0.01)}")
    print(f"Expected: x=2, y=1, λ=-4, f=6")

### Exercise 4: Optimization Landscape Explorer

Write a function that takes a 2D objective function and visualizes:
1. The contour plot
2. The gradient descent path from a given starting point
3. The final (approximate) optimum

In [ ]:
# EXERCISE 4: Build an optimization landscape explorer
def explore_landscape(f, grad_f, x0, lr=0.01, n_steps=200,
                      x_range=(-5, 5), y_range=(-5, 5)):
    """
    Visualize a 2D optimization landscape with gradient descent path.
    
    Args:
        f: Objective function f(x, y) -> scalar
        grad_f: Gradient function grad_f(x, y) -> (df/dx, df/dy)
        x0: Starting point [x, y]
        lr: Learning rate
        n_steps: Number of gradient descent steps
        x_range: (min, max) for x axis
        y_range: (min, max) for y axis
    """
    # TODO: Implement this!
    # Step 1: Create meshgrid and compute Z = f(X, Y)
    # Step 2: Run gradient descent, storing the path
    # Step 3: Plot contours + path + start/end markers
    # Hint: Use ax.contourf for filled contours and ax.plot for the path
    
    pass  # Replace with your solution

# Test with Rosenbrock function (a classic test function)
f_rosen = lambda x, y: (1 - x)**2 + 100*(y - x**2)**2
grad_rosen = lambda x, y: (
    -2*(1 - x) - 400*x*(y - x**2),
    200*(y - x**2)
)

print("Implement explore_landscape() to visualize gradient descent on the")
print("Rosenbrock function. The minimum is at (1, 1).")
print("Try different learning rates and starting points!")

# Uncomment after implementing:
# explore_landscape(f_rosen, grad_rosen, x0=[-1, -1], lr=0.001, n_steps=5000,
#                   x_range=(-2, 2), y_range=(-1, 3))

---

## Summary

### Key Concepts

- **Optimization** is the process of finding the best solution from a feasible set. It requires an objective function, decision variables, and (optionally) constraints.
- **Linear programming** optimizes a linear objective subject to linear constraints. The solution always occurs at a vertex of the feasible polytope.
- **The Simplex method** solves LPs by hopping between adjacent vertices, always improving the objective.
- **Duality** provides an alternative view of any LP. Strong duality means primal and dual have the same optimal value.
- **Convexity** is the key property that guarantees local minima are global minima. Most classical ML algorithms are convex.
- **Lagrange multipliers** solve equality-constrained problems by requiring gradient alignment at the optimum.
- **KKT conditions** generalize Lagrange multipliers to inequality constraints. Complementary slackness identifies active constraints.
- **ML is optimization**: SVMs solve QPs, regularization is dual to constraints, and loss functions are designed to be convex.

### Connection to Machine Learning

| Concept from this Notebook | Where You Will See It in ML |
|---------------------------|-----------------------------|
| Objective function | Every loss function (MSE, cross-entropy, hinge) |
| Feasible region / constraints | Regularization ($\ell_1$, $\ell_2$), fairness constraints |
| Convexity | Guarantees for logistic regression, SVM, LASSO |
| Non-convexity | Neural network training challenges |
| Lagrange multipliers / KKT | SVM support vectors, dual formulations |
| Duality | Kernel trick in SVMs, ELBO in variational inference |
| Linear programming | Resource allocation, network optimization |
| Gradient descent (preview) | The topic of the next notebook! |

### Checklist

Before moving on, make sure you can:

- [ ] Define an optimization problem with objective, variables, and constraints
- [ ] Formulate an LP in standard form and solve it with `linprog`
- [ ] Explain why LP solutions occur at vertices
- [ ] Describe the Simplex method's vertex-hopping strategy
- [ ] State weak and strong duality theorems
- [ ] Distinguish convex from non-convex functions
- [ ] Explain why convexity guarantees a global optimum
- [ ] Set up and solve a Lagrangian for equality constraints
- [ ] State the four KKT conditions and explain complementary slackness
- [ ] Frame an SVM as a constrained optimization problem
- [ ] Explain regularization as a Lagrangian constraint

---

## Next Steps

Continue to **Part 1.6: Gradient-Based Optimization** where we will cover:
- Gradient descent and its variants (SGD, momentum, Adam)
- Learning rate schedules and convergence theory
- Automatic differentiation
- How these methods actually train neural networks

**Looking ahead:** Now that you understand *what* optimization is and *when* it works well (convexity), the next notebook focuses on *how* we actually solve optimization problems in practice — with gradients. Every deep learning training loop is gradient descent applied to a (usually non-convex) loss function.